### Load packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
import yfinance as yf

from finrl.meta.data_processor import DataProcessor 
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent 
from stable_baselines3.common.logger import configure
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from argparse import ArgumentParser 

import itertools
import pyfolio
from pyfolio import timeseries

/home/dz/.local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


### Create folders

In [2]:
from finrl import config
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [3]:
TRAIN_START_DATE = '2015-01-01'
TRAIN_END_DATE = '2020-01-01'
TEST_START_DATE = '2020-01-02'
TEST_END_DATE = '2022-12-01'
TRADE_START_DATE = '2022-12-01'
TRADE_END_DATE = '2023-04-01'
TIME_INTERVAL = '1D'

ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', 
               '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', 
               '600519.SH', '600547.SH', '600570.SH']


### Download and Preprocess Data

In [4]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Add Indicator to Data

In [5]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2075, 8)
Successfully added vix
Successfully added turbulence index


Reformat Data and write to csv file

In [6]:
# get the only tic and date
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

# get the processed full data
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

# fill the missing values with zero
processed_full = processed_full.fillna(0)

# sort the data and reset index
processed_full.sort_values(['date','tic'],ignore_index=True)

# write to csv file
processed_full.to_csv("dow_30.csv",index=False)

Read Data from csv file (optional)

In [7]:
processed_full = pd.read_csv("dow_30.csv")

In [8]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2015-01-02,AAPL,27.847500,27.860001,26.837500,24.531765,212818400.0,4.0,0.0,25.163582,23.208843,0.0,-66.666667,100.0,24.531765,24.531765,17.790001,0.0
1,2015-01-02,AMGN,160.160004,162.589996,158.600006,127.031990,2605400.0,4.0,0.0,25.163582,23.208843,0.0,-66.666667,100.0,127.031990,127.031990,17.790001,0.0
2,2015-01-02,AXP,93.169998,93.940002,92.139999,81.897881,2437500.0,4.0,0.0,25.163582,23.208843,0.0,-66.666667,100.0,81.897881,81.897881,17.790001,0.0
3,2015-01-02,BA,131.070007,131.839996,129.089996,113.657204,4294200.0,4.0,0.0,25.163582,23.208843,0.0,-66.666667,100.0,113.657204,113.657204,17.790001,0.0
4,2015-01-02,CAT,91.769997,92.370003,90.660004,72.221130,3767900.0,4.0,0.0,25.163582,23.208843,0.0,-66.666667,100.0,72.221130,72.221130,17.790001,0.0


### Build Environment

Splite the data into training set and testing set.

In [9]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
test = data_split(processed_full, TEST_START_DATE,TEST_END_DATE)

Calculate the stock dimension and state space.

In [10]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [11]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

env_train = StockTradingEnv(df = train, **env_kwargs)

### Train DRL Agents

In [12]:
agent = DRLAgent(env = env_train)

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

# set up logger
tmp_path = RESULTS_DIR + "/ppo"
new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to results/ppo


In [13]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 1.26e+03  |
|    ep_rew_mean     | 57.2      |
| time/              |           |
|    fps             | 266       |
|    iterations      | 1         |
|    time_elapsed    | 7         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 0.3605704 |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.26e+03     |
|    ep_rew_mean          | 55.4         |
| time/                   |              |
|    fps                  | 241          |
|    iterations           | 2            |
|    time_elapsed         | 16           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.017304406  |
|    clip_fraction        | 0.195        |
|    clip_range           | 0.2          |
|    entropy_loss    